In [ ]:
# pip install openai langchain neo4j speechrecognition pandas
# pip install ipykernel
# pip install langchain-openai
# pip install --upgrade langchain langchain-community langchain-openai
# pip install python-dotenv

In [3]:
# ai_policy_compliance.ipynb

import openai
import neo4j
import langchain
import speech_recognition as sr
import pandas as pd
from neo4j import GraphDatabase
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.vectorstores import Neo4jVector

In [4]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Access API keys securely
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USER = os.getenv("NEO4J_USER")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

print(f"Connected to Neo4j at: {NEO4J_URI}")

Connected to Neo4j at: None
